In [49]:
import sys
import os

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.append(project_root)
 
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [50]:
import pandas as pd
import numpy as np

import proyeccion_rdr.features as features
from proyeccion_rdr.produccion.a04_ambulatorio import (
    obtener_distribucion_consultas,
    procesar_incidencias,
    leer_consultas_no_medicas,
    expandir_serie_rendimientos,
)

pd.set_option("display.max_columns", None)

ANIOS_POBLACION = [str(i) for i in range(2017, 2036)]

In [51]:
# Lee los casos de todos los macroprocesos
RUTA_ARCHIVOS = "../data/interim/0.1_casos_teoricos_diagnosticos.xlsx"
casos_macroproceso_por_region, casos_macroproceso = features.leer_casos_macroprocesos(
    RUTA_ARCHIVOS
)

# Pone el indice en el DataFrame de los casos
casos_macroproceso = casos_macroproceso.reset_index().set_index(["Diagnostico", "tipo_paciente"])
casos_macroproceso_por_region = casos_macroproceso_por_region.reset_index().set_index(
    ["Diagnostico", "tipo_paciente"]
)

# Define las areas de influencia presenciales
AREAS_PRESENCIALES = [
    "Metropolitana de Santiago",
    "COMUNAS_SIN_SS_EN_RM",
    "SSMC",
    "SSMN",
    "SSMO",
    "SSMOC",
    "SSMS",
    "SSMSO",
    "Acotado por oferta",
]
casos_macroproceso_por_region["es_presencial"] = np.where(
    casos_macroproceso_por_region["Estrato"].isin(AREAS_PRESENCIALES),
    "Box Presencial",
    "Box de Telemedicina",
)

## Objetivo de este cuadernillo

En este cuadernillo se quieren estimar la cantidad de pacientes ambulatorios que asisitrán al INT para el 2035. Este insumo se utilizará para estimar la cantidad de Box Ambulatorios para cada especialidad. Por lo tanto, es necesario hacer los siguientes calculos:

1. Pacientes Ambulatorios
2. Boxes Ambulatorios

In [52]:
RUTA_INCIDENCIAS = (
    "../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx"
)

(
    df_incidencias,
    diagnosticos_ingresados,
    casos_por_especialidad_long,
    casos_totales_por_especialidad_y_grupo,
    casos_a_hacerse_cargo_long,
    casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    casos_a_hacerse_cargo_consolidados,
) = procesar_incidencias(
    RUTA_INCIDENCIAS,
    "consultas_no_medicas_grupo_de_p",
    casos_macroproceso_por_region,
    ANIOS_POBLACION,
)

Iniciando proceso...
Cargando datos desde ../data/raw/3_incidencias_y_porcentajes_marcoprocesos/incidencias_y_prevalencias_RDR.xlsx, hoja: consultas_no_medicas_grupo_de_p
Separando diagnósticos por especialidad...
Diagnósticos únicos ingresados: 16
Indexando por tipo de paciente y diagnóstico...
Uniendo casos por diagnóstico con macroprocesos...
Controlando las trazadoras ingresadas...
Todas las trazadoras tienen casos asociados. No se requiere control adicional.
Identificando casos duplicados...
Especialidades con diagnósticos duplicados: 

Series([], Name: Diagnostico, dtype: object)

Sumando pacientes por grupo y especialidad...
Calculando casos a hacerse cargo...
Sumando pacientes por grupo y especialidad...
Sumando pacientes por grupo y especialidad...
Proceso completado.


## Test de Calidad

En este test se quiere saber si todas las trazadoras estan ingresadas en el macroproceso ambulatorio

In [53]:
# Lee la planilla de trazadoras y aisla
trazadoras_totales = pd.read_excel(RUTA_INCIDENCIAS, sheet_name="trazadoras")
trazadoras_totales = set(trazadoras_totales["Diagnostico"].str.split(" - ").str[0].unique())

In [54]:
# diagnosticos_sin_ingresar = trazadoras_totales - diagnosticos_ingresados
# if len(diagnosticos_sin_ingresar) > 0:
#     raise ValueError(f"Falta ingresar las trazadoras: {diagnosticos_sin_ingresar}")

## Lectura de performance historico de ambulatorio

In [55]:
# Lee las consultas medicas
RUTA_AMBULATORIO = "../data/raw/6_ambulatorio/df_procesada_consultas_no_medicas.csv"
consultas_no_medicas = leer_consultas_no_medicas(RUTA_AMBULATORIO)

C:\Users\javie\OneDrive\Documents\Trabajo\SSMN\2.0-proyeccion-rdr\proyeccion_rdr\produccion\a04_ambulatorio.py:110: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(ruta, dtype={"id_paciente": str})


In [56]:
# Obtiene resumen de consultas acumuladas en el periodo
agrupacion_acumulada = ["especialidad_agrupada"]
distribucion_consultas_no_medicas_acumuladas, consultas_no_medicas_por_paciente_acumuladas = (
    obtener_distribucion_consultas(consultas_no_medicas, agrupacion_acumulada)
)

# Aisla el nombre de la columna que tenga el 75% de las consultas
columna_estadistica_cantidad_consultas = distribucion_consultas_no_medicas_acumuladas.columns[
    distribucion_consultas_no_medicas_acumuladas.columns.str.contains("75%")
][0]

# Indica cuantas consultas por cada una de las especialidades
cantidad_consultas_no_medicas_a_ocupar = distribucion_consultas_no_medicas_acumuladas[
    columna_estadistica_cantidad_consultas
]

# Filtra las especialidades para solo dejar las que estan en la cartera de servicios
cantidad_consultas_no_medicas_a_ocupar = cantidad_consultas_no_medicas_a_ocupar[
    cantidad_consultas_no_medicas_a_ocupar.index.isin(
        casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.index.get_level_values(0)
    )
]

In [57]:
cantidad_consultas_no_medicas_a_ocupar

especialidad_agrupada
ENFERMERÍA              2.0
NUTRICIÓN               8.0
PSICOLOGÍA              7.0
QUÍMICA Y FARMACIA      2.0
TERAPIA OCUPACIONAL    12.0
TRABAJO SOCIAL          4.0
Name: 75%_entre_2015_2024, dtype: float64

## Definicion de desempeno para consultas sin produccion

In [58]:
# Define la cantidad de consultas que tendran las especialidades sin un rendimiento historico
consultas_sin_desempeno = pd.Series(
    {
        "PSICOPEDAGOGÍA": None,
        "PSIQUIATRÍA PEDIATRICA Y DE LA ADOLESCENCIA": None,
    }
)

# Concatena las especilidades que si tienen desempeno con las que no las tienen
cantidad_consultas_no_medicas_a_ocupar = pd.concat(
    [cantidad_consultas_no_medicas_a_ocupar, consultas_sin_desempeno]
)

In [59]:
cantidad_consultas_no_medicas_a_ocupar

ENFERMERÍA                                      2.0
NUTRICIÓN                                       8.0
PSICOLOGÍA                                      7.0
QUÍMICA Y FARMACIA                              2.0
TERAPIA OCUPACIONAL                            12.0
TRABAJO SOCIAL                                  4.0
PSICOPEDAGOGÍA                                 None
PSIQUIATRÍA PEDIATRICA Y DE LA ADOLESCENCIA    None
dtype: object

In [60]:
# Indica la cantidad de consultas que tendran la telemedicina
grupos_de_pacientes = [1, 2, 3]
valores_es_presencial = ["Box Presencial", "Box de Telemedicina"]

# Expande la cantidad de consultas segun grupo de pacientes y valores presenciales
cantidad_consultas_no_medicas_a_ocupar = expandir_serie_rendimientos(
    cantidad_consultas_no_medicas_a_ocupar, grupos_de_pacientes, valores_es_presencial
)

In [61]:
TIEMPO_CONSULTA = {
    "15 minutos": 0.25,
    "30 minutos": 0.5,
    "45 minutos": 0.75,
    "60 minutos": 1,
    "20 minutos": 0.333,
    "40 minutos": 0.666,
}

rendimientos_reales = pd.Series(
    {
        "ENFERMERÍA": TIEMPO_CONSULTA["45 minutos"],
        "NUTRICIÓN ": TIEMPO_CONSULTA["45 minutos"],
        "PSICOLOGÍA": TIEMPO_CONSULTA["45 minutos"],
        "QUÍMICA Y FARMACIA": TIEMPO_CONSULTA["45 minutos"],
        "TERAPIA OCUPACIONAL": TIEMPO_CONSULTA["45 minutos"],
        "TRABAJO SOCIAL": TIEMPO_CONSULTA["45 minutos"],
        "PSICOPEDAGOGÍA": TIEMPO_CONSULTA["45 minutos"],
        "PSIQUIATRÍA PEDIATRICA Y DE LA ADOLESCENCIA": TIEMPO_CONSULTA["45 minutos"],
    }
)

# Expande los rendimientos
rendimientos_reales = expandir_serie_rendimientos(
    rendimientos_reales, grupos_de_pacientes, valores_es_presencial
)

## Estimacion de Consultas al 2035

In [62]:
# Obtiene la cantidad de horas laborales
horas_laborales = features.calcular_horas_laborales(2017, 2035, 12)

Horas laborales por año calculadas:
+---+------+---------------------------------+
|   | anio | horas_laborales_solo_semana_12h |
+---+------+---------------------------------+
| 0 | 2017 |              2976               |
| 1 | 2018 |              2964               |
| 2 | 2019 |              2988               |
| 3 | 2020 |              3024               |
| 4 | 2021 |              3012               |
+---+------+---------------------------------+



In [63]:
# Multiplica los casos de area de influencia
consultas_proyectadas = casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial.mul(
    cantidad_consultas_no_medicas_a_ocupar, axis=0
).dropna(axis=0, how="all")

# Multiplica la cantidad de consultas por el tiempo que requerira cada consulta
tiempo_consultas = consultas_proyectadas.mul(rendimientos_reales, axis=0).dropna(axis=0, how="all")

# Divide la cantidad de consultas por especialidad, por el rendimiento de cada box
boxes_proyectados = tiempo_consultas.div(horas_laborales, axis=1)

In [64]:
# Obtiene el resumen de Boxes por estamnento
boxes_consolidados = (
    boxes_proyectados.reset_index()
    .groupby("ESTAMENTO/ESPECIALIDAD")
    .sum()["2035"]
    .sort_values(ascending=False)
)

# Agrega el total
boxes_consolidados["Total"] = boxes_consolidados.sum()

In [65]:
display(boxes_consolidados)

ESTAMENTO/ESPECIALIDAD
TERAPIA OCUPACIONAL    35.497825
PSICOLOGÍA              7.768977
QUÍMICA Y FARMACIA      4.556072
ENFERMERÍA              3.738329
TRABAJO SOCIAL          0.425515
Total                  51.986718
Name: 2035, dtype: object

In [66]:
# Obtiene la cantidad de boxes medicos y de telemedicina
boxes_presencial_y_tele = (
    boxes_proyectados.reset_index().groupby("es_presencial")[ANIOS_POBLACION].sum()
)

In [67]:
boxes_presencial_y_tele

,2017,2018,2019,2020,2021,2022,2023,2024,2025,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035
es_presencial,,,,,,,,,,,,,,,,,,,
Box Presencial,40.497664,41.968395,42.885359,43.5589,44.424877,45.04778,46.019172,46.087225,46.488029,46.420853,46.606779,47.269524,47.690711,47.341545,47.001641,46.478111,46.830837,47.494332,47.610884
Box de Telemedicina,4.811625,4.808808,4.774126,4.734353,4.774565,4.819983,4.904078,4.896944,4.928109,4.895387,4.862613,4.854846,4.813839,4.688399,4.576091,4.457363,4.42334,4.419828,4.375834


In [68]:
# Define columnas que se utilizar para generar resumen MINSAL
cols_indice = ["ESTAMENTO/ESPECIALIDAD", "Grupos de Pacientes", "es_presencial"]

# Genera un DataFrame de las trazadoras con presencialidad
df_incidencias_resumen = df_incidencias.copy()
df_incidencias_resumen["es_presencial"] = "Box Presencial, Box de Telemedicina"
df_incidencias_resumen["es_presencial"] = df_incidencias_resumen["es_presencial"].str.split(", ")
df_incidencias_resumen = df_incidencias_resumen.explode("es_presencial")
df_incidencias_resumen = df_incidencias_resumen.set_index(cols_indice)

## Resumen MINSAL

In [69]:
# Obtiene el resumen MINSAL
resumen_MINSAL = pd.DataFrame(
    {
        "tipo_paciente": df_incidencias_resumen["tipo_paciente"],
        "Diagnostico": df_incidencias_resumen["Diagnostico"],
        "Areas de Influencia a atender presencial": str(AREAS_PRESENCIALES),
        "numero_de_pacientes_totales_2035": casos_totales_por_especialidad_y_grupo["2035"],
        "% de los pacientes a atender": df_incidencias_resumen["% de los pacientes a atender"],
        "explicacion_pacientes": df_incidencias_resumen["Explicación Pacientes"],
        "casos_a_hacerse_cargo_2035": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial[
            "2035"
        ],
        "consultas_por_paciente_75%": cantidad_consultas_no_medicas_a_ocupar,
        "consultas_proyectadas_2035": consultas_proyectadas["2035"],
        "horas_por_consultas": rendimientos_reales,
        "horas_consultas_2035": tiempo_consultas["2035"],
        "horas_laborales_2035": horas_laborales["2035"],
        "boxes_proyectados_2035": boxes_proyectados["2035"],
    }
)

resumen_MINSAL = resumen_MINSAL.dropna().reset_index(level=[1, 2])

In [70]:
a_guardar = {
    "resumen_MINSAL": resumen_MINSAL,
    "boxes_consolidados": boxes_consolidados,
    "trazadoras_ambulatorio": df_incidencias,
    "casos_macroprocesos_por_region": casos_macroproceso_por_region.reset_index(),
    "casos_por_esp_long": casos_por_especialidad_long.reset_index(),
    "casos_a_hacerse_cargo_long": casos_a_hacerse_cargo_long,
    "casos_por_esp_grupo_y_pres": casos_a_hacerse_cargo_por_especialidad_grupo_y_presencial,
    "casos_consolidados": casos_a_hacerse_cargo_consolidados,
    "metricas_historicas_por_espec": distribucion_consultas_no_medicas_acumuladas,
    "rendimientos_utilizados": rendimientos_reales,
    "consultas_medicas_proyectadas": consultas_proyectadas,
    "tiempo_por_especialidad": tiempo_consultas,
    "horas_laborales": horas_laborales,
    "boxes_medicos_proyectados": boxes_proyectados,
    "boxes_presenciales_y_tele": boxes_presencial_y_tele,
}

with pd.ExcelWriter("../data/interim/3.1_estimacion_boxes_no_medicos_RDR.xlsx") as file:
    for nombre_hoja, df_a_guardar in a_guardar.items():
        df_a_guardar.to_excel(file, sheet_name=nombre_hoja)